In [1]:
import os 
import pandas as pd
import tensorflow as tf
import numpy as np

In [2]:
df = pd.read_csv("train.csv")

In [3]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [4]:
df.iloc[7]

id                                                00031b1e95af7921
comment_text     Your vandalism to the Matt Shirvington article...
toxic                                                            0
severe_toxic                                                     0
obscene                                                          0
threat                                                           0
insult                                                           0
identity_hate                                                    0
Name: 7, dtype: object

In [5]:
df['insult'].value_counts()

0    151694
1      7877
Name: insult, dtype: int64

In [6]:
#PREPROCESSING

In [7]:
from tensorflow.keras.layers import TextVectorization

In [8]:
X = df['comment_text']
y = df[df.columns[2:]].values

In [9]:
max_features = 200000 #this means the total number of words the machine will select from

In [26]:
vectorizer = TextVectorization(max_tokens=max_features,
                               output_sequence_length=1800,
                               output_mode='int')

In [27]:
vectorizer.adapt(X.values)

In [28]:
vectorized_text = vectorizer(X.values)

In [29]:
vectorized_text

<tf.Tensor: shape=(159571, 1800), dtype=int64, numpy=
array([[  643,    76,     2, ...,     0,     0,     0],
       [    1,    54,  2506, ...,     0,     0,     0],
       [  425,   440,    70, ...,     0,     0,     0],
       ...,
       [32141,  7329,   383, ...,     0,     0,     0],
       [    5,    12,   533, ...,     0,     0,     0],
       [    5,     8,   130, ...,     0,     0,     0]], dtype=int64)>

In [14]:
len(X)

159571

In [30]:
#MSCHBAP: Map, chache, shuffle, batch, prefetch,
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text, y))
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8) #helps stop obttleneck stuff

In [16]:
#dataset.as_numpy_iterator().next()

In [17]:
len(dataset)

9974

In [31]:
train = dataset.take(int(len(dataset)*.7))
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))

In [19]:
#SEQUENTIAL MODEL

In [32]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding

In [21]:
y.shape

(159571, 6)

In [33]:
model = Sequential()
# create the embedding layer, +1 is for seperate unknown words
model.add(Embedding(max_features+1, 32))
# create the bidirectional layer push info in 2 ways, using tanh for better preformance and becuase its a LSTM  and helps with NLP
model.add(Bidirectional(LSTM(32, activation='tanh')))
# bunch of dense layers, feature extactores, fully connected layers 
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
# this is the final layer, maps the y values and output the same number of outputs as training data
# sigmoid because we want values between 0 and 1
model.add(Dense(6, activation='sigmoid'))

In [34]:
model.compile(loss='BinaryCrossentropy', optimizer='Adam')

In [35]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 32)          6400032   
                                                                 
 bidirectional_1 (Bidirectio  (None, 64)               16640     
 nal)                                                            
                                                                 
 dense_4 (Dense)             (None, 128)               8320      
                                                                 
 dense_5 (Dense)             (None, 256)               33024     
                                                                 
 dense_6 (Dense)             (None, 128)               32896     
                                                                 
 dense_7 (Dense)             (None, 6)                 774       
                                                      

In [36]:
history = model.fit(train, epochs=1, validation_data=val)

6981/6981 [==============================] - 3128s 448ms/step - loss: 0.0634 - val_loss: 0.0476


In [3]:
from keras.models import load_model

In [4]:
history.save("toxic_model.h5")

NameError: name 'history' is not defined

In [ ]:
print(hello)